Predictive Trading Algorithm using TensorFlow

- Install/Import our packages
- Set up model parameters
- Retrieve some data using Finnhub API
- Setup the MACross Backtrading strategy in Backtrader
- Find optimal parameters and get a baseline performance.

Install and Import Packages

In [1]:
# Install Needed Packages. If using Google Collab these need to be in seperate cells.
#!pip install finnhub-python
#!pip install keras-tuner
#!pip install backtrader
#!pip install pandas
#!pip install matplotlib

# Import needed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.markers as mark
import tensorflow as tf
import sys 
from sklearn.preprocessing import MinMaxScaler
import math
from pandas_datareader import data as web
import finnhub
from datetime import datetime
from datetime import timezone
from time import time, sleep
import requests
from google.colab import drive
from google.colab import files
import seaborn as sns

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow import keras
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch 
from kerastuner.tuners import Hyperband 
from pylab import rcParams

rcParams['figure.figsize'] = 16,9
rcParams['figure.facecolor'] = '#eeeeee'
plt.title('dummy')
plt.plot([1,3,2,4])
plt.close()

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import backtrader as bt
import argparse
import backtrader.feeds as btfeeds
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Have all of our plots show automatically
%matplotlib inline

# Mount your Google Drive to the worksheet
drive.mount('/content/drive')

/Users/henrik/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


ModuleNotFoundError: No module named 'tensorflow'

Model Parameters

In [ ]:
# Global Parameters
ticker = "CVX"
granularity = "D"

# The amount of time divisions the LSTM model will look back for predictions
lookback_distance = 60

# Time Period for Stratagy Tuning
dt_start_model_parameter_tuning = datetime(2000, 10, 19)
dt_end_model_parameter_tuning = datetime(2006, 12, 31)

# Time Period for Model Building
dt_start_nn_model_training = datetime(2007, 1,1)
dt_end_nn_model_training = datetime(2013, 12, 1)

# Time Period for Backtesting
dt_start_backtest = datetime(2014, 1,1)
dt_end_backtest = datetime(2020, 12, 9)

# SMA Cross Stratagy Search Grid. This defines the windows we will test to find an optimal 
# fast and slow window combination.
pslow_start  = 30;
pslow_end    = 200;
pslow_step   = 10;

pfast_start  = 30;
pfast_end    = 200;
pfast_step   = 10;

# API Keys. Go to the finnhub website to get your API key and put it here.
finnhub_api_key = "Your Key Here";

# Drive Path
google_drive_path = "/content/drive/My Drive/"

Retrieve and Setup Data

In [ ]:
# Covert the dates to UTC timestamps that will be used by the Finnhub API call
timestamp_start = int(dt_start_model_parameter_tuning.replace(tzinfo=timezone.utc).timestamp())
timestamp_end = int(dt_end_model_parameter_tuning.replace(tzinfo=timezone.utc).timestamp())

# Initialize the Client
finnhub_client = finnhub.Client(api_key=finnhub_api_key)

# Get data and put into dataframe.
res = finnhub_client.stock_candles(ticker, granularity, timestamp_start, timestamp_end)
data = pd.DataFrame(res)

# Make Data Backtrader Friendly
dataBacktrader = data
dataBacktrader.columns = ['close', 'high', 'low','open','s','t','volume']

dataBacktrader['date'] = dataBacktrader.apply(lambda row : datetime.utcfromtimestamp(row['t']), axis = 1)
dataBacktrader.drop(['s','t'], axis = 1, inplace=True)
dataBacktrader['openinterest'] = 0

# Plot the data
plt.plot(dataBacktrader['close'])

Setup SmaCross Model

Strategy already constructed in the Backtrader quick start page. 
One line added which checks correct parameters otherwise it stops the run to not waste compute time.

In [ ]:
# Create a subclass of Strategy to define the indicators and logic
class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=60,  # period for the fast moving average window
        pslow=128   # period for the slow moving average window
    )
    
    # The __init__ method runs at the start of any stratagy.
    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        
        # Only want to test valid parameters (and workable parameters)
        # otherwise skip the run
        if ((self.p.pfast <= self.p.pslow) and (self.p.pslow - self.p.pfast >= 5)) :
          self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
        else :
          raise bt.StrategySkipError   

    # The next() method is run each step of the stratagy. 
    # The step of a stratagy is determined by the dataframe and the duraction 
    # between each data point.
    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position
            
    # The stop method runs at the end of a stratagy run. 
    def stop(self):
        print('(FA Period %2d) (SL Period %2d) Ending Value %.2f' %
                 (self.params.pfast, self.params.pslow, self.broker.getvalue()))   

Run SMACross Strategy and Baseline Performance

Build and run a Cereboro object so that Backtrader will run our strategy over a set of data.

In [ ]:
# Create array to store the results from all the backtrader runs.
results_list = []

# First we want to initialize our Cerebro object.
cerebro = bt.Cerebro(stdstats=False,maxcpus=None)

# Next we set the amount of cash we want to start out with.
cerebro.broker.setcash(100000.0)

# Next we read in the dataframe with the data we want backtest
dataFrame = bt.feeds.PandasData(dataname=dataBacktrader, datetime='date')
cerebro.adddata(dataFrame)

# Now we load the actual stratagy. Since we want to run the stratagy over a large
# combination of parameters we want to use the optstratagy method. We
# define the range of parameters we want to run over based on the additional arguments.
# Here we are running our stratagy over:
# - pFast = pfast_start to pfast_end, with a stepsize of pfast_step
# - pSlow = pslow_start to pslow_end, with a stepsize of pslow_step
cerebro.optstrategy(SmaCross, pslow=range(pslow_start,pslow_end+1,pslow_step), pfast=range(pfast_start,pfast_end+1,pfast_step))

# Next we add a sizer to the cerebro object. This determines the strategy of how large our buy and 
# sell orders are. Here we fix it top 99% of our portfolio. 
cerebro.addsizer(bt.sizers.PercentSizer, percents=99)

# We are almost there but we now need to add analyzers to the cerebro object.
# These analyzers allow us to extract information about how well the stratagy performed. 
# (Links to more info about these analyzers at the bottom of the post)
cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name="sharperatio", timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.NoTimeFrame)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")

# Finally we want to set how much each tranasction will cost in commision fees. Since I personally
# use Robinhood Ill just set this to 0.
cerebro.broker.setcommission(commission=0)

# Finally we run the model. This will run the stratagy over the wide array of parameters
# we set above. An object is returned which holds all of the results of the runs.
# (And the information we told it to gather from our Analyzers)
results = cerebro.run();

# Next we want to run over this results object and extract the information so it is easily
# readable and indexable. 
for i in range(0, len(results)):  
  # Need to check that this is a run that fully ran
  if (len(results[i]) >= 1) :
    # Temporarily store all the analyzer values into variables. 
    r_strat  = "slow_fast_" + str(results[i][0].params.pslow) + "_" + str(results[i][0].params.pfast);
    r_return = list(results[i][0].analyzers.timereturn.get_analysis().values())[0]*100
    r_sharpe = results[i][0].analyzers.sharperatio.get_analysis()['sharperatio'];
    r_draw   = results[i][0].analyzers.drawdown.get_analysis()['drawdown']/100;
    r_sqn   = results[i][0].analyzers.sqn.get_analysis()['sqn']

    # Append a dictionary which holds all the important results into our results array.
    results_list.append({
       "Stratagy" : r_strat,
       "pslow" : results[i][0].params.pslow,
       "pfast" : results[i][0].params.pfast,
       "Return" : r_return,
       "Sharpe" : r_sharpe,
       "SQN" : r_sqn,
       "Drawdown" : r_draw
    })


# Make the results a Pandas dataframe and give us statsitical information 
# about the values of the runs. 
results = pd.DataFrame(results_list)               
results.describe()


In [ ]:
#        pslow       pfast       Return      Sharpe      SQN          Drawdown
#count   153.000000  153.000000  153.000000  153.000000  153.000000  153.000000
#mean    146.666667  83.333333   55.581273   0.499984    0.710518    0.052910
#std     41.231056   41.231056   20.770793   0.156723    0.353140    0.045569
#min     40.000000   30.000000   3.309254    0.048054    -0.034536   0.031449
#25%     120.000000  50.000000   40.986669   0.393614    0.472027    0.031853
#50%     150.000000  80.000000   53.110563   0.485314    0.658597    0.031938
#75%     180.000000  110.000000  69.834780   0.604150    0.960126    0.044747
#max     200.000000  190.000000  126.805272  0.983955    1.684354    0.262238

Values:

- pslow - duration of the slow-moving average window'
- pfast - duration of the fast-moving average window
- Return - Percentage return of our strategy.
- Annualized Sharpe - A metric that takes into account the return above, the returns from a risk-free investment, and the standard deviation of our returns. The Sharpe ratio gives us a better idea of how good a strategy is as it takes into account the risk and volatility of a strategy. I would suggest reading up some more on the Sharpe ratio.
- SQN — This number takes into account the average profit from a trade, the variability of the profit, and how many trades you did. (From Backtrader Docs ->) System Quality Number defined by Van K. Tharp to categorize trading systems.
1.6 - 1.9 Below average
2.0 - 2.4 Average
2.5 - 2.9 Good
3.0 - 5.0 Excellent
5.1 - 6.9 Superb

Create a heatmap of the Sharpe values to see any patterns
Sharpe takes into account not just the return but also the variation in return.

In [ ]:
# Set up data frame where the rows are pslow and the columns are pfast.
heat_df = pd.DataFrame(columns=results['pfast'].drop_duplicates(keep='first', inplace=False))
row = results['pslow'].drop_duplicates(keep='first', inplace=False)
heat_df['pslow'] = row
heat_df = heat_df.set_index('pslow')
heat_df = pd.DataFrame(columns=results['pfast'].drop_duplicates(keep='first', inplace=False))
row = results['pslow'].drop_duplicates(keep='first', inplace=False)
heat_df['pslow'] = row
heat_df = heat_df.set_index('pslow')

heat_df.columns.name = None
heat_df.index.names = ['']

# Go through all the combinations of pfast and pslow and put 
# in the Sharpe ratio of the run with those params.
pslowCols = results.filter(['pslow','pfast','Sharpe'])
for index, row in pslowCols.iterrows():
    heat_df.loc[row['pslow'],row['pfast']] = float(row['Sharpe'])

# I needed to change the type of data in the dataframe from
# object to float. 
heat_df = heat_df.astype(np.float16)

# Display heat map
sns.heatmap(heat_df)

There can be times the best value is surrounded by a bunch of combinations that have negative Sharpe ratios. If this is the case, either modify the range, switch the value you optimize for, or select another local max that’s in a more stable region.

Setting up and Training the LSTM Model
(data from Finnhub)

In [ ]:
timestamp_start = int(dt_start_nn_model_training.replace(tzinfo=timezone.utc).timestamp())
timestamp_end = int(dt_end_nn_model_training.replace(tzinfo=timezone.utc).timestamp())

finnhub_client = finnhub.Client(api_key=finnhub_api_key)
res = finnhub_client.stock_candles(ticker, granularity, timestamp_start, timestamp_end)
data = pd.DataFrame(res)

Get optimized pfast and pslow values from above
Create two new columns in our data set which correspond to the fast and slow-moving average indicators.

Training and Testing Data

In [ ]:
fast_data = data.filter(['fast_ra'])
slow_data = data.filter(['slow_ra'])

# Get Number of Rows to train model. (Use 80%)
fast_training_data_len = math.ceil( len(fast_data) * .8)
# Create train and test data arrays.
fast_train_data = fast_data[0:fast_training_data_len]
fast_test_data = fast_data[fast_training_data_len-lookback_distance:]

# Get Number of Rows to train model. (Use 80%)
slow_training_data_len = math.ceil( len(slow_data) * .8)
# Create train and test data arrays.
slow_train_data = slow_data[0:slow_training_data_len]
slow_test_data = slow_data[slow_training_data_len-lookback_distance:]

Scale the data using a MinMaxScalar. This modifies the data so the minimum value is 0 and the maximum value is 1. Then all values in-between will be linearly fit to a value between 0 and 1.
This makes it easier for the model to fit and train.

In [ ]:
#Scale the Data
fast_scalar = MinMaxScaler(feature_range=(0,1))
fast_scaled_train_data = fast_scalar.fit_transform(fast_train_data)
fast_scaled_test_data = fast_scalar.fit_transform(fast_test_data)

#Scale the Data
slow_scalar = MinMaxScaler(feature_range=(0,1))
slow_scaled_train_data = slow_scalar.fit_transform(slow_train_data)
slow_scaled_test_data = slow_scalar.fit_transform(slow_test_data)

two models, one for the fast-moving average and one for the slow-moving average. This means we need two identical sets of code to set up the models

create two arrays for each model. One array where each entry is an array of 60 moving average values (x), and another array which is the target moving average (y).

For example:

The first entry of the x array will be [ma0, ma1, ma2, … ma59] and the first entry of the y array is ma60

The second entry of the x array will be [ma1, ma2, ma3, … ma60] and the first entry of the y array is ma61

The pattern is we get 60 ma values to predict the 61st ma value.

In [ ]:
fast_x_train = []
fast_y_train = []

slow_x_train = []
slow_y_train = []

# Create the two arrays. One with the set of 60 MA values leading to the target MA
# and one with the target MA
# We need to start at our lookahead distance to have enough values for the first 
# target moving average. 
for i in range(lookback_distance, len(fast_scaled_train_data)):
 fast_x_train.append(fast_scaled_train_data[i-lookback_distance:i, 0])
 fast_y_train.append(fast_scaled_train_data[i, 0])
 slow_x_train.append(slow_scaled_train_data[i-lookback_distance:i, 0])
 slow_y_train.append(slow_scaled_train_data[i, 0])

# Convert x_train and y_train to numpy array
fast_x_train,  fast_y_train = np.array(fast_x_train), np.array(fast_y_train)
slow_x_train,  slow_y_train = np.array(slow_x_train), np.array(slow_y_train)

# Reshape Data into Samples, Timesteps, Features
fast_x_train = np.reshape(fast_x_train, (fast_x_train.shape[0] ,fast_x_train.shape[1], 1))
slow_x_train = np.reshape(slow_x_train, (slow_x_train.shape[0] ,slow_x_train.shape[1], 1))

Train the Model using Karas
First a static model for both the fast and slow-moving averages with an initial set of parameters. The model will look like:

(60 Previous MA Values) -> LSTM Layer -> LSTM Layer -> Dense Hidden Layer -> Single Neuron Output -> (Predicted MA)

In [ ]:
# Initialize the constructor
best_fast_model = Sequential()
# Add an first hidden layer 
best_fast_model.add(LSTM(30, return_sequences=True, input_shape = (fast_x_train.shape[1],1)))
# Add a second hidden layer 
best_fast_model.add(LSTM(30, return_sequences=False))
# Add a third hidden layer 
best_fast_model.add(Dense(8))
# Add an output layer with one neuron and no activation specified
best_fast_model.add(Dense(1))

# Initialize the constructor
best_slow_model = Sequential()
# Add an first hidden layer 
best_slow_model.add(LSTM(30, return_sequences=True, input_shape = (slow_x_train.shape[1],1)))
# Add a second hidden layer 
best_slow_model.add(LSTM(30, return_sequences=False))
# Add a third hidden layer 
best_slow_model.add(Dense(8))
# Add an output layer with one neuron and no activation specified
best_slow_model.add(Dense(1))

opt = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile Model
best_fast_model.compile(optimizer=opt,loss='mean_squared_error', metrics=['accuracy','mse'])
best_slow_model.compile(optimizer=opt,loss='mean_squared_error', metrics=['accuracy','mse'])

A Nerul Net is trained by taking a sample from the training set, inputting it into a randomly initialized Neural Net, and receiving an output. You then find that difference between the output and the target variable to determine the error. The computer then propagates backward through the model trying to correct for this error by modifying the weights and threshold of the neurons. This is done multiple times over all the training data to build an accurate model. One forward and backward pass over all the training data is called an epoch. One forward and backward pass over a subset of training data is called an iteration. The size of the subset of data in an iteration is called the batch size.

model.fit() Inputs:

an array of leading ma values (X)
an array of target ma values (y)
batch_size - The amount of training data used for each iteration of a forward and backward pass. (Higher the more memory is needed for each pass)
epochs - The number of epochs to train the model. (Higher the better fit)
validation_split- The amount of data that will be set aside and not be used for training. This data will be used to evaluate the specified metrics at the end of each epoch.

In [ ]:
# Train the model
best_fast_model.fit(fast_x_train, fast_y_train, batch_size=1, epochs=4, validation_split=0.1)
best_slow_model.fit(slow_x_train, slow_y_train, batch_size=1, epochs=4, validation_split=0.1)
Epoch 4/4 ... mse: 1.8177e-04 - val_loss: 1.4620e-04 - val_accuracy: 0.0083 - val_mse: 1.4620e-04

Epoch 4/4 ... mse: 1.3387e-04 - val_loss: 1.1088e-05 - val_accuracy: 0.0083 - val_mse: 1.1088e-05

One of the things we want to look at is the MSE is Mean Squared Errors. This is a measurement that tells us how far our model is off on average. Its units are the same as the target variable squared. (Dollars^2 in this case). Here we see the mse is approximately the val_mse. This is important as if the mse of the training set is significantly lower than validation we are probably overfitting meaning our model is fitting to much to the noise. If it's much higher than we are probably underfitting which is not fitting to the data well enough.

After this step, we have two fully functioning models that can predict each of the fast and slow-moving averages. However, there are a lot of numbers and parameters that we just set randomly or by default when building the model. It would be useful if we could run the model over a whole set of parameters and see what the optimal set of parameters are. This is called hyperparameter optimization.

Train the Model using karas-tuner

We are going to use a package called karas-tuner to tune our model. To set it up we are going to make a HyperModel which is just the model we had before, however we replace some of the parameters with ranges of parameters we can test.

In [ ]:
class HyperModel(HyperModel):
  # When making the model its possible to have meta parameters such as levels and shape.
  # These come in when initializing the HyperModel and here we have 2:
  # - The depth of the model. (We are just using 1 for now but thought it would be useful to show)
  # - The shape of the input data.
  def __init__(self,levels,input_shape) :
    self.levels      = levels;
    self.input_shape = input_shape;

  # Here we define our model just like above.
  def build(self, hp) :
    model = Sequential()
    # But here we replace 30 with this object which says we want to test values 28 to 36
    model.add(LSTM(
          units = hp.Int(
            'units',
            min_value=28,
            max_value=36,
            step=2,
            default=32
        ),
        return_sequences=True, 
        input_shape = self.input_shape
        )
    )

    # But here we replace 30 with this object which says we want to test values 28 to 36
    model.add(LSTM(
          units = hp.Int(
            'units',
            min_value=28,
            max_value=36,
            step=2,
            default=32
        ),
        return_sequences=False
        )
    )

    # Here we have a loop which can create multiple hidden layers according to levels
    for i in range(0, self.levels):
      # Here we replace 8 with this object which says we want to test values 6 to 10
      model.add(Dense(
        units = hp.Int(
            'units',
            min_value=6,
            max_value=10,
            step=2,
            default=8
        ),
        # Here we replace the default activation with other activation functions. 
        activation=hp.Choice(
            'dense_activation',
            values=['relu', 'tanh', 'sigmoid','linear'],
            default='linear'
        )
      ))

    # Here we replace the default activation with other activation functions.     
    model.add(Dense(1,
        activation=hp.Choice(
            'dense_activation',
            values=['relu', 'tanh', 'sigmoid','linear'],
            default='linear'
        )
    ))
    
    # We can also optimize parameters to the compile command. Here we test differant learning rates. 
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float(
                'learning_rate',
                min_value=8e-4,
                max_value=1e-3,
                sampling='LOG',
                default=1e-3
            )
        ),
        loss='mean_squared_error', 
        metrics=['mse', 'accuracy']
    )

    return model

# Create fast and slow models from same general HyperModel
fasthypermodel = HyperModel(1, (fast_x_train.shape[1],1));
slowhypermodel = HyperModel(1, (slow_x_train.shape[1],1));

Next we set up the tuner. The tuner determines how we will search the parameter space as we cannot possibly test all combinations. For example, one option is Random which will just test a certain amount of random combinations, however here we select a search method with some more smarts, Hyperband.

We can have it search as short or as long as we want, however, the longer you have it run the better results you will get.

In [ ]:
# Once you run there will be a dialogue showing what iteration you are on.
# To get an estimate of how many iterations will be run use this formula: 
# hyperband_iterations * max_epochs * (math.log(max_epochs, factor) ** 2)

# Note: I had to change project name each time I ran. The good news is that you can
# run this, stop it, and then continue where it left off as it continuously saves your model. 
# You can even load a model from before.

# It took about 2 hrs to get a result with these settings. 

fast_tuner = Hyperband(
    fasthypermodel,
    objective='mse',
    seed=5354,
    max_epochs=4,
    hyperband_iterations=5,
    factor = 2,
    executions_per_trial=1,
    directory=google_drive_path+'hyperparameter_tuning',
    project_name='name0'
)

slow_tuner = Hyperband(
    slowhypermodel,
    objective='mse',
    seed=9953,
    max_epochs=4,
    hyperband_iterations=5,
    factor = 2,
    executions_per_trial=1,
    directory=google_drive_path+'hyperparameter_tuning',
    project_name='name1'
)

# (Optional) These output the search space the hyper tuner will run.
fast_tuner.search_space_summary()
slow_tuner.search_space_summary()

After we set up the tuners we just have to run them. They take the same arguments as the .fit() method.

In [ ]:
fast_tuner.search(fast_x_train, fast_y_train, batch_size=1, epochs=4,validation_split=0.1) 
slow_tuner.search(slow_x_train, slow_y_train, batch_size=1, epochs=4,validation_split=0.1)
# (Optional) Show a summary of the search
fast_tuner.results_summary()
slow_tuner.results_summary()

# Retrieve the best model.
best_opt_fast_model = fast_tuner.get_best_models(num_models=1)[0]
best_opt_slow_model = slow_tuner.get_best_models(num_models=1)[0]

# (Optional) Save Models
best_fast_model.save(google_drive_path+"fast_model")
best_slow_model.save(google_drive_path+"slow_model")
best_opt_fast_model.save(google_drive_path+"fast_opt_model")
best_opt_slow_model.save(google_drive_path+"slow_opt_model")

At this point we have four total models. Two we created without any optimization and two which were optimized with hyperparameter tuning. Now we need to create the test set the same way we created the training set. Then get predictions, grab the RSME (Square Root of MSE), and plot the difference to see how the models did.

In [ ]:
# Create the testing set
fast_x_test = []
fast_y_test = fast_data[fast_training_data_len:]

slow_x_test = []
slow_y_test = slow_data[slow_training_data_len:]

for i in range(lookback_distance, len(fast_scaled_test_data)):
 fast_x_test.append(fast_scaled_test_data[i-lookback_distance:i, 0])
 slow_x_test.append(slow_scaled_test_data[i-lookback_distance:i, 0])

# Convert the data to a numpy
fast_x_test = np.array( fast_x_test)
slow_x_test = np.array( slow_x_test)

# Reshape Data
fast_x_test = np.reshape( fast_x_test, ( fast_x_test.shape[0] , fast_x_test.shape[1], 1))
slow_x_test = np.reshape( slow_x_test, ( slow_x_test.shape[0] , slow_x_test.shape[1], 1))

# Preditions. We have to remember the model takes 
# in Scaled data so we have to un scale it coming out. 
predictions_opt_fast = best_opt_fast_model.predict(fast_x_test)
predictions_opt_fast = fast_scalar.inverse_transform(predictions_opt_fast)

predictions_opt_slow = best_opt_slow_model.predict(slow_x_test)
predictions_opt_slow = slow_scalar.inverse_transform(predictions_opt_slow)


# Preditions. We have to remember the model takes 
# in Scaled data so we have to un scale it coming out. 
predictions_fast = best_fast_model.predict(fast_x_test)
predictions_fast = fast_scalar.inverse_transform(predictions_fast)

predictions_slow = best_slow_model.predict(slow_x_test)
predictions_slow = slow_scalar.inverse_transform(predictions_slow)

# Evaluate Model and get RMSE
print("Hyperparameter Tuned")
fast_opt_rmse = np.sqrt( np.mean( predictions_opt_fast - fast_y_test )**2 )
slow_opt_rmse = np.sqrt( np.mean( predictions_opt_slow - slow_y_test )**2 )
print(fast_opt_rmse)
print(slow_opt_rmse)

# Evaluate Model and get RMSE
print("Untuned")
fast_rmse = np.sqrt( np.mean( predictions_fast - fast_y_test )**2 )
slow_rmse = np.sqrt( np.mean( predictions_slow - slow_y_test )**2 )
print(fast_rmse)
print(slow_rmse)
Hyperparameter Tuned
fast_ra    0.313589
dtype: float64
slow_ra    0.083455
dtype: float64
Untuned
fast_ra    0.151402
dtype: float64
slow_ra    0.090759
dtype: float64

Here we calculate RSME as it is easier to interpret as its in units of what we are predicting. (Dollars in this case).

The approximate MSEs in this case are above what we got in Training and Validation which may point to overfitting. To account for this we could do things such as lowering the Neuron count of each layer.

Here I was getting RSMEs smaller than a dollar, but the RSMEs may change depending on how that specific stock behaves. In addition, you need to take into consideration how expensive the stock is to evaluate this value since it's not normalized. For example, being 20 cents off a 5$ stock is a lot worse than 20 cents off a $300 stock.

Now we want to plot the data.

In [ ]:
ftrain = fast_data[:fast_training_data_len]
fvalid = fast_data[fast_training_data_len:]
fvalid['predictions'] = predictions_fast

strain = slow_data[:fast_training_data_len]
svalid = slow_data[fast_training_data_len:]
svalid['predictions'] = predictions_slow

plt.figure(figsize=(16,8))
plt.plot(ftrain['fast_ra'])
plt.plot(strain['slow_ra'])

plt.plot(fvalid[['fast_ra','predictions']])
plt.plot(svalid[['slow_ra','predictions']])

Retrieve and Setup Data

Same stuff. Different date range.

In [ ]:
# Validation for PFast and PSlow Parameters of MA Crossover
timestamp_start = int(dt_start_backtest.replace(tzinfo=timezone.utc).timestamp())
timestamp_end = int(dt_end_backtest.replace(tzinfo=timezone.utc).timestamp())

finnhub_client = finnhub.Client(api_key=finnhub_api_key)
res = finnhub_client.stock_candles(ticker, 'D', timestamp_start, timestamp_end)
data = pd.DataFrame(res)
data.dropna(inplace=True)

# Make Data Backtrader Friendly
dataBacktrader = data
dataBacktrader.columns = ['close', 'high', 'low','open','s','t','volume']

dataBacktrader['date'] = dataBacktrader.apply(lambda row : datetime.utcfromtimestamp(row['t']), axis = 1)
dataBacktrader.drop(['s','t'], axis = 1, inplace=True)
dataBacktrader['openinterest'] = 0

Create Pandas Dataframe with Indicators

Now we want to create a dataframe that has all of our indicators included that Backtrader can run. Here we want to add 6 columns to our typical dataframe:

fast_ra - Fast Rolling Average
slow_ra - Slow Rolling Average
pred_fast_opt - Tomorrow's predicted fast ra from the tuned model.
pred_slow_opt - Tomorrow's predicted fast ra from the tuned model.
pred_fast - Tomorrow's predicted fast ra from the untuned model.
pred_slow - Tomorrow's predicted fast ra from the untuned model.

In [ ]:
# Populate the data frame with the rolling averages
data['fast_ra'] = data.iloc[:,1].rolling(window=opt_fast).mean()
data['slow_ra'] = data.iloc[:,1].rolling(window=opt_slow).mean()
data.dropna()

fast_data = data.filter(['fast_ra'])
slow_data = data.filter(['slow_ra'])

# Here we do the same thing as when we constructed  the testing set.
# Instead we will use all of the data in the backtesting data set.
fast_x_test = []
fast_y_test = fast_data[lookback_distance:]

slow_x_test = []
slow_y_test = slow_data[lookback_distance:]

fast_scaled_test_data = fast_scalar.fit_transform(fast_data)
slow_scaled_test_data = slow_scalar.fit_transform(slow_data)

for i in range(lookback_distance, len(fast_scaled_test_data)):
 fast_x_test.append(fast_scaled_test_data[i-lookback_distance:i, 0])
 slow_x_test.append(slow_scaled_test_data[i-lookback_distance:i, 0])

# Convert the data to a numpy
fast_x_test = np.array( fast_x_test)
slow_x_test = np.array( slow_x_test)

# Reshape Data
fast_x_test = np.reshape( fast_x_test, ( fast_x_test.shape[0] , fast_x_test.shape[1], 1))
slow_x_test = np.reshape( slow_x_test, ( slow_x_test.shape[0] , slow_x_test.shape[1], 1))

# Preditions. We have to remember the model takes 
# in Scaled data so we have to un scale it coming out. 
predictions_opt_fast = best_opt_fast_model.predict(fast_x_test)
predictions_opt_fast = fast_scalar.inverse_transform(predictions_opt_fast)

predictions_opt_slow = best_opt_slow_model.predict(slow_x_test)
predictions_opt_slow = slow_scalar.inverse_transform(predictions_opt_slow)

# Preditions. We have to remember the model takes 
# in Scaled data so we have to un scale it coming out. 
predictions_fast = best_fast_model.predict(fast_x_test)
predictions_fast = fast_scalar.inverse_transform(predictions_fast)

predictions_slow = best_slow_model.predict(slow_x_test)
predictions_slow = slow_scalar.inverse_transform(predictions_slow)

# Evaluate Model and get RMSE of Backtesting Set
print("Hyperparameter Tuned")
fast_opt_rmse = np.sqrt( np.mean( predictions_opt_fast - fast_y_test )**2 )
slow_opt_rmse = np.sqrt( np.mean( predictions_opt_slow - slow_y_test )**2 )
print(fast_opt_rmse)
print(slow_opt_rmse)

# Evaluate Model and get RMSE of Backtesting Set
print("Untuned")
fast_rmse = np.sqrt( np.mean( predictions_fast - fast_y_test )**2 )
slow_rmse = np.sqrt( np.mean( predictions_slow - slow_y_test )**2 )
print(fast_rmse)
print(slow_rmse)
Hyperparameter Tuned
fast_ra    0.617608
dtype: float64
slow_ra    0.477331
dtype: float64
Untuned
fast_ra    0.168646
dtype: float64
slow_ra    0.025827
dtype: float64

Here we see the RSME grew even more which makes sense, however it is still a relatively low value. I just used the RSMEs here as a checkpoint to make sure the models are correctly trained and fit.

Now to construct the actual DataFrame.

In [ ]:
# Construct dataframe that will feed Backtrader Stratagy
dataBacktrader_trunc = dataBacktrader.loc[lookback_distance:,:]
dataBacktrader_trunc['pred_fast_opt'] = predictions_opt_fast
dataBacktrader_trunc['pred_slow_opt'] = predictions_opt_slow
dataBacktrader_trunc['pred_fast'] = predictions_fast
dataBacktrader_trunc['pred_slow'] = predictions_slow
dataBacktrader_trunc.dropna(inplace=True)

plt.figure(figsize=(16,8))
plt.plot(dataBacktrader_trunc[['fast_ra','pred_fast_opt','pred_fast']])
plt.plot(dataBacktrader_trunc[['slow_ra','pred_slow_opt','pred_slow']])

Since we added indicators to our dataframe we need to tell Backtester about how to read it in. To do this we will create a new class to take in the dataframe object.

In [ ]:
# Here we define the fields of our custom dataframe object that will feed Backtrader
class MAData(btfeeds.PandasData):

    # What new data will be availible in the Stratagies line object
    lines = ('fast_ra', 'slow_ra', 'pred_fast_opt', 'pred_slow_opt', 'pred_fast', 'pred_slow')

    # Which columns go to which variable
    params = (
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', 'openinterest'),
        ('fast_ra', 7),
        ('slow_ra', 8),
        ('pred_fast_opt', 9),
        ('pred_slow_opt', 10),
        ('pred_fast', 11),
        ('pred_slow', 12)
    )

Now we want to create our custom strategy that implements our models. You could also go the route of creating custom-built indicators but that could be for another time.

In [ ]:
# Create a subclass of Strategy to define the indicators and logic
class SmaCrossPredicted(bt.Strategy):
    # We have a single parameter which indicates whther we want to use the:
    # 0 - Non Hyperparameter Tunbed Model
    # 1 - Hyperparameter tuned model.
    params = dict(
        opt=0
    )

    # At initialization of the stratagy
    def __init__(self):
        # We only need to setup the cross indicator of the predicted values. 
        self.crossover = bt.ind.CrossOver(self.data.l.pred_fast_opt,                                            self.data.l.pred_slow_opt)  

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position

    def stop(self):
        print("...",self.p.opt)

Backtest and Evaluate Stratagies

Now using our custom strategy we want to backtest three versions:

opt - The tuned model
non opt - The untuned model
no model - The Classic SMACross strategy

In [ ]:
# Arrays to store result data
results_list   = []
results_struct = []
strat          = ["Tuned Model", "Non Tuned Model", "Classic MA Cross"]

# Arrays to store buy, sell tranactions
txns           = [[],[],[]]

# Go through all of the stratagies in the strat array
for i in range(0,3):
  # Same initialization as before
  cerebro = bt.Cerebro(stdstats=False,maxcpus=None)
  cerebro.broker.setcash(100000.0)

  dataFrame = MAData(dataname=dataBacktrader_trunc, datetime='date')
  cerebro.adddata(dataFrame)

  # Depending on strat add the correct stratagy 
  if (i == 0) :
    cerebro.addstrategy(SmaCrossPredicted, opt = 1)  
  elif (i == 1) :
    cerebro.addstrategy(SmaCrossPredicted,  opt = 0)  
  else :
    cerebro.addstrategy(SmaCross, pslow=opt_slow, pfast=opt_fast)

  # Same as before
  cerebro.addsizer(bt.sizers.PercentSizer, percents=99)
  cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Days)
  cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.NoTimeFrame)
  cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
  cerebro.addanalyzer(bt.analyzers.DrawDown)
  cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")

  # Here we add a Transaction analyzer so we can see the differance of the stratagies
  # in terms of transaction start and lengths.
  cerebro.addanalyzer(bt.analyzers.Transactions, _name="txn")

  cerebro.broker.setcommission(commission=0)
  results = cerebro.run();

  # Now we have to accumulate the results ourselves 
  # vs all of the results being returned in one cerebro run.
  results_list.append(results)

  # We need to modify the transactions from dates of buys and sells to arrays of
  # (Date of Buy, How long it was held). This helps us to plot the transaction lengths
  for item in results_list[i][0].analyzers.txn.get_analysis().items():
    if (item[1][0][0] < 0):
       sell_date = int(item[0].replace(tzinfo=timezone.utc).timestamp())   
       txns[i].append((start_date,sell_date-start_date)) 
    else : 
       start_date = int(item[0].replace(tzinfo=timezone.utc).timestamp())

# Go through results and create a results dataframe.
for i in range(0, len(results_list)):    
  r_strat  = strat[i];
  r_return = list(results_list[i][0].analyzers.timereturn.get_analysis().values())[0]*100
  r_sharpe = results_list[i][0].analyzers.sharperatio.get_analysis()['sharperatio'];
  r_draw   = results_list[i][0].analyzers.drawdown.get_analysis()['drawdown']/100;
  r_sqn    = results_list[i][0].analyzers.sqn.get_analysis()['sqn']

  results_struct.append({
      "Stratagy" : r_strat,
      "Return" : r_return,
      "Sharpe" : r_sharpe,
      "Drawdown" : r_draw,
      "SQN" : r_sqn
  })

resultspd = pd.DataFrame(results_struct)    

for i in range(0, 3):   
  # See References and Resources
  printTradeAnalysis(results_list[i][0].analyzers.ta.get_analysis())

resultspd

#Output:
#Tuned Trade Analysis Results:
#               Total Open     Total Closed   Total Won      Total Lost     
#               0              3              2              1              
#               Strike Rate    Win Streak     Losing Streak  PnL Net        
#               66.666666666666662              1              38933.27       
#Untuned Trade Analysis Results:
#               Total Open     Total Closed   Total Won      Total Lost     
#               0              7              3              4              
#               Strike Rate    Win Streak     Losing Streak  PnL Net        
#               42.8571428571428542              4              31463.47       
#MA Cross Trade Analysis Results:
#               Total Open     Total Closed   Total Won      Total Lost     
#               0              7              3              4              
#               Strike Rate    Win Streak     Losing Streak  PnL Net        
#               42.8571428571428542              4              21216.55       

#            Stratagy         Return     Sharpe      Drawdown    SQN
#0           Tuned Model      38.933272  0.033459    0.055129    1.342116
#1           Non Tuned Model  31.463466  0.020075    0.054352    0.946327
#2           Classic MA Cross 21.216555  0.013938    0.083485    0.811401

Now we create a broken bar plot to visualize when each strategy held positions in the stock and might provide insight into why our model did better.

In [ ]:
fig, ax = plt.subplots(figsize=(30, 10))
ax.broken_barh(txns[0], (10, 9), facecolors='gray', hatch = 'X')
ax.broken_barh(txns[1], (20, 9), facecolors='gray', hatch = '/')
ax.broken_barh(txns[2], (30, 9), facecolors='gray', hatch = '.')
plt.ylabel("Tuned Model                                   Untuned Model                              MACross")
for i in range(0,len(txns[2])):
  ax.axvline(x=txns[2][i][0], color="black")
plt.show()